# Detection of tree stems with Pyoints
In the following, we try to detect stems in a forest using a three dimnesional point cloud generated by a terrestrial laser scanner.The basic idea is to
1. Load the data 

In [ ]:
from pyoints import (
	storage,
	filters,
	interpolate,
)

In [ ]:
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt 

%matplotlib inline

## Loading of data

Load the a LAS file of a forest.

In [ ]:
lasReader = storage.LasReader('forest.las')

Get some information of the point cloud.

In [ ]:
print('number of points:')
print(len(lasReader))
print('projection:')
print(lasReader.proj.proj4)
print('transformation matrix:')
print(lasReader.t)
print('origin:')
print(lasReader.t.origin)
print('extent:')
print(lasReader.extent)

If everything is fine we load the points.

In [ ]:
las = lasReader.load()

We recieve a numpy record array of the point cloud. So we inspect its properties first.

In [ ]:
print('shape:')
print(las.shape)
print('attributes:')
print(las.dtype)
print('projection:')
print(las.proj.proj4)
print('transformation:')
print(las.t)
print('data')
print(las)

## Generation of height values
Since we LAS file does just provide altitude values instead of heights above ground, we select points representing the ground first, to fit a digital elevation model (DEM). Using the DEM we calculate the heiht of each point above ground.

We use a DEM filter of a resolution of 0.5 m. The filter selects low points and garanties a horizontal point distance of at least 0.5 m and a maximal altitude change between neighbored points of 50 degree.

In [ ]:
grd_ids = filters.dem_filter(las.coords, 0.5, max_angle=50)
print(grd_ids)

We have recieved a list of point indices, which can be used to select the desired representative ground points. So we plot them first.

In [ ]:
x = las.coords[grd_ids, 0]
y = las.coords[grd_ids, 1]
plt.scatter(x, y)
plt.xlabel('X (m)')
plt.ylabel('Y (m)')
plt.show()


In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')
coords = las.coords[grd_ids, :]
plt.scatter(coords[:, 0], coords[:, 1], coords[:, 2])
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_zlabel('Z (m)')
plt.show()


We can see that the points are distributed almost uniformly.

Now we fit a DEM using a nearest neigbour interpolator.

In [ ]:
xy = las.coords[grd_ids, :2]
z = las.coords[grd_ids, 2]
dem = interpolate.KnnInterpolator(xy, z)

Finally we calculate the height above ground.

In [ ]:
height = las.coords[:, 2] - dem(las.coords)